In [ ]:
################################################
# Welcome to Module 6: Data Analysis in Pandas

# -- Contents --
# 1. DATA DESCRIPTIONS AND AGGREGATIONS
# 2. GROUPED AGGREGATIONS
# 3. SORTING AND FILTERING
# 4. TABLE JOINS
# 4. ADVANCED STATISTICAL METHODS
# 5. TRANSFORMING DATAFRAMES
# 6. VISUALIZING DATA
# 7. ACTIVITIES

################################################


########################
# 1. DATA DESCRIPTIONS AND AGGREGATIONS
########################


##########
## 1.1 SETUP
##########

In [ ]:
# First, we'll import several libraries
import pandas as pd
import pickle

In [ ]:
# Then, we'll load the 2 dataframes that we'll be working with
# The first is the same sales dataset that we worked with in Module 5

sales_df = pd.read_pickle('./data/sales_df.pickle')
customer_df = pd.read_pickle('./data/customers_data.pickle')

In [ ]:
# Also the same as Module 5, we'll force Pandas to show all of the columns when printing
pd.set_option('display.max_columns', 1000) # Now we will see up to 1000 columns
pd.set_option('display.max_colwidth', 1000) # And we will see up to 1000 characters in each column

##########
## 1.2 DATA DESCRIPTIONS
##########

In [ ]:
# We can use the .describe() function to view some general statistics about each column in a dataframe
# Let's look at the sales dataframe
sales_description = sales_df.describe()
sales_description

In [ ]:
# Now let's look at the customer dataframe
cust_description = customer_df.describe()
cust_description

In [ ]:
# To determine the data type of each column, we can use the .dtypes function
# Note: Strings will show as 'object'
dtypes = sales_df.dtypes
dtypes

##########
## 1.3 AGGREGATIONS
##########

In [ ]:
# There are many built-in aggregation function we can use to compute individual column statistics
# These include, but are not limited to: count, sum, mean, median, mode, min, max, abs, prod, std, var 
# Note: Some of these statistics are the same as what we see when we use the .describe() function

# We can use .count() to tell us the number of rows in each column containing non-NA values

sales_df.count()

In [ ]:
# For all of these aggregation functions, we can also apply them to a subset of columns

# selection = sales_df[ ['customer_key', 'product_key'] ]
# count = selection.count()
# print(count)

sales_df[['customer_key', 'product_key']].count()

In [ ]:
# We can use .sum() to sum all of the values in each column
selection = sales_df['product_cost']
my_sum = selection.sum() # Note: We cannot call our variable 'sum' because that is a reserved Python keyword
print(my_sum)

In [ ]:
# We can also use the .agg() function to compute multiple statistics on 1 or more columns at once
# Note: This is similar to what the .describe() function does, but it allows us to specify the stats we want
stats = sales_df['product_cost'].agg(['count', 'sum', 'mean', 'std', 'min', 'max'])
print(stats)

In [ ]:
# Exercise
'''
Find the average product_cost, average order_quantity, and mean product_price
'''

########################
# 2. GROUPED AGGREGATIONS
########################

##########
## 2.1 BASIC GROUPED AGGREGATIONS
##########

In [ ]:
# Rather than computing column statistics that consider each row individually, 
#   we can group rows based on one of their column values using the .groupby() function
# Then, we can compute statistics for each group

# We'll start with a simple example: grouping and computing a single metric
grouped_data = sales_df.groupby('product_name')
count = grouped_data['order_quantity'].agg('count') # Count number of rows in each group with non-NA values for 'OrderQuantity'
count

In [ ]:
# Exercise
'''
Using .loc -

Find the order_quantity count for 'Classic Vest, M'

Hint: DataFrame indices can be found with `df_name.index`
'''

count.index

In [ ]:
# We can accomplish the same thing in a more compact way
count = sales_df.groupby('product_name')['order_quantity'].agg('count')
print(count)

In [ ]:
# Similar to what we saw in the previous section, we can compute multiple metrics at once
# Group and compute
stats = sales_df.groupby('product_name')['order_quantity'].agg(['count', 'sum', 'mean', 'std', 'min', 'max']) 
stats.head()

In [ ]:
# Exercise
'''
Using .loc -

Find the max order_quantity for 'Bike Wash - Dissolver'

Hint: DataFrame columns can be found with `df_name.columns`
'''

In [ ]:
stats.index

In [ ]:
# We can use .reset_index() to move the 'product_name' from the index back to a regular column,
# and set the new row numbers as the index.
stats = stats.reset_index()
stats.head()

In [ ]:
# Exercise
'''
From sales_df
1. Find the average product_cost for each product_name.
2. Find the sum of product_cost for 'AWC Logo Cap'
'''

##########
## 2.2 ADVANCED GROUPED AGGREGATIONS
##########

In [ ]:
# For more complex aggregations, it's best to use your own aggregation function

# Here, we define a function that computes the 90th percentile value for a given data series
import numpy as np
def ninetieth_percentile(x):
  p = np.percentile(x, 90)
  return(p)

In [ ]:
# Then, we can supply the function name as an input in the .agg() function
stats = sales_df.groupby('category_name')['product_price'].agg(['min', ninetieth_percentile, 'max']).reset_index() 
stats

########################
# 3. SORTING AND FILTERING
########################

##########
## 3.1 SORTING
##########

In [ ]:
# We can sort rows using the .sort_values() function
# Note: If sorting using multiple columns, the function first sorts by the left-most column provided
#       Then, rows are further sorted by the next column provided, etc.
#       This can handle cases where some column values are the same.
sorted_df = sales_df.sort_values(by = ['stock_date', 'product_name', 'product_size'], ascending=True) 

sorted_selection = sorted_df[['stock_date', 'product_name', 'product_size', 'model_name']]

sorted_selection.head()

In [ ]:
sorted_df = sorted_df.reset_index(drop = True)
sorted_df.head()

In [ ]:
##########
# 3.2 FILTERING
##########
# Filtering rows is the equivalent of using `WHERE` in SQL.
# Note: Filtering occurs by requesting rows where a condition is `True`.

# First, we need to build a series of True/False values for each row 
# The value is true for each row where column 'CategoryName' has value 'Bikes'
wanted_category_name = 'Bikes' 
rows_tf = (sales_df['category_name'] == wanted_category_name)
rows_tf.head(20)

In [ ]:
# Next, we use the True/False series to filter the dataframe
# The result of this funtion is a new dataframe with all of the rows that meet our filter condition
filtered_df = sales_df[rows_tf]
print('The original dataframe has %d rows' % len(sales_df))
print('The filtered dataframe has %d rows' % len(filtered_df))

filtered_df[['product_name', 'model_name', 'product_style']].head()

In [ ]:
# Exercise
'''
Find all rows with product cost > 2000
How many rows are there?
'''

In [ ]:
# We can accomplish the same filtering in a more compact way
wanted_category_name = 'Bikes' 
columns_wanted = ['product_name', 'model_name', 'product_style', 'subcategory_name']
filtered_df = sales_df[sales_df['category_name'] == wanted_category_name][columns_wanted]

print('The filtered dataframe has %d rows' % len(filtered_df))
filtered_df.head()

In [ ]:
# We can also filter using compound conditions
wanted_category_name = 'Bikes' 
subcategory_name = 'Mountain Bikes'

columns_wanted = ['product_name', 'model_name', 'product_style', 'subcategory_name']

rows_tf = (sales_df['category_name'] == wanted_category_name) & (sales_df['subcategory_name'] == subcategory_name)

filtered_df = sales_df[rows_tf][columns_wanted]

print('The filtered dataframe has %d rows' % len(filtered_df))
filtered_df.head()

In [ ]:
# Another type of filtering is considering only unique values, using the .drop_duplicates() function
# Here, we first sort the dataframe using the 'ProductName' and 'ModelName' columns

sorted_df = sales_df.sort_values(by = ['product_name', 'model_name'], ascending=False) 
# print(sorted_df[['product_name', 'model_name', 'product_style']].head())

# Then, we drop all rows containing duplicate values in columns 'ProductName' and 'ModelName'
unique_sorted_df = sorted_df.drop_duplicates(['product_name', 'model_name'])

# Now, let's look at the result
unique_sorted_df[['product_name', 'model_name', 'product_style']].head()

In [ ]:
unique_sorted_df[['product_name', 'model_name', 'product_style']].reset_index(drop = True).head()

In [ ]:
# Exercise
'''
Find all rows with category = Bikes, subcategory_name == Mountain Bikes, and product_cost between 1500 and 2000
'''

########################
# 4. TABLE JOINS
########################

https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins <br>
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html


##########
## 4.1 GETTING STARTED
##########

In [ ]:
import pandas as pd
import numpy as np
import pickle
sales_df = pd.read_pickle('./data/sales_df.pickle')
customer_df = pd.read_pickle('./data/customers_data.pickle')

In [ ]:

# Although the sales data (sales_df) has more rows than the customer data (customer_df), 
# the former has fewer unique customer_key's than the latter.

# inner and outer joins

# Sales data
print(len(sales_df['customer_key']))
print(len(sales_df['customer_key'].drop_duplicates()))

# Customer data
print(len(customer_df['customer_key']))
print(len(customer_df['customer_key'].drop_duplicates()))

In [ ]:
sales_df.loc[:5, 'customer_key']

In [ ]:
customer_df.loc[:5, 'customer_key']

In [ ]:
x = np.asarray(sales_df.columns.to_list())
y = np.asarray(customer_df.columns.to_list())

np.intersect1d(x, y)

##########
## 4.2 INNER JOIN
##########

In [ ]:
# Get the intersection of the Sales and customer entries with an identical customer_key

joined_df = pd.merge(   left=sales_df, right=customer_df,                 # Left and right dataframes
                        how='inner',                                      # Specify inner join
                        left_on='customer_key', right_on='customer_key',  # Join keys
                        suffixes=('_sales', '_cust')                      # To apply to overlapping column names
                    )

print('The joined df has %d unique customer_key values' % len(joined_df['customer_key'].drop_duplicates()))
joined_df.head()

##########
## 4.3 LEFT JOIN
##########

In [ ]:
# "Complement" the Sales data with the Customer dataset
# Some sales entries do not have a counterpart in the cust data. Look at the `_merge` column added by the 
# argument 'indicator=True' to see which rows from the sales data did not have a counterpart in the customer dataset
joined_df = pd.merge(   left=sales_df, right=customer_df, 
                        how='left',
                        left_on='customer_key', right_on='customer_key',
                        suffixes=('_sales', '_cust'),
                        indicator=True)
print('The joined df has %d unique customer_key values' % len(joined_df['customer_key'].drop_duplicates()))
joined_df.head()

##########
## 4.4 RIGHT JOIN
##########

In [ ]:
# For each entry in the customer dataset, find all the sales which match on customer_key
# Look at the _merge column added by the argument 'indicator=True' 
#  to see which customer rows did not have a counterpart in the sales dataset.
joined_df = pd.merge(   left=sales_df, right=customer_df, 
                        how='right',
                        left_on='customer_key', right_on='customer_key',
                        suffixes=('_sales', '_cust'),
                        indicator=True)
print('The joined df has %d unique customer_key values' % len(joined_df['customer_key'].drop_duplicates()))
joined_df.head()

In [ ]:
# Exercise
'''
1. Find rows in the df created after right join that are only present in customer_df

HINT : _merge can have the values - 'both', 'left_only', 'right_only'

'''

##########
## 4.5 OUTER JOIN
##########

In [ ]:
# Keep all the customer_key's from both sales and customer datasets and attmpt to match each of them (left + right join).
# Look at the '_merge' column added by the argument 'indicator=True' to see which rows did not have a counterpart 
# in the other dataset.
joined_df = pd.merge(   left=sales_df, right=customer_df, 
                        how='outer',
                        left_on='customer_key', right_on='customer_key',
                        suffixes=('_sales', '_cust'))
print('The joined df has %d unique customer_key values' % len(joined_df['customer_key'].drop_duplicates()))
joined_df.head()

########################
# 5. ADVANCED STATISTICAL METHODS
########################

##########
## 5.1 LAMBDA FUNCTIONS
##########

In [ ]:
# Lambda functions are single-line functions that typically perform some time of data transformation 
# They can be used inside the .apply() function to very efficiently perform the same operation
# on each row in a dataframe column

# For example, the following lambda function is used to add 'NEW--' to the beginning of each value in the 'OrderNumber' column
# The new values are stored in a new column named 'new_OrderNumber'
sales_df['new_order_number'] = sales_df['order_number'].apply(func=(lambda x: 'NEW--' + str(x)))
sales_df[['order_number', 'new_order_number']].head()

In [ ]:
# We can also use lambda functions that act on multiple dataframe columns
# Can you tell what the following function will do before executing the code?
sales_df['new_order_name_date'] = sales_df.apply(lambda row: ' - '.join([str(row.order_number), str(row.order_date)]), axis=1)
sales_df[['order_number', 'order_date', 'new_order_name_date']].head()

In [ ]:
# For more complex tranformations, its best to create a normal function that is called within the lambda function
# Here, we define the function
def concatenate_num_date(order_num, order_date):
  concatenated = ''.join(['NEW--', str(order_num), '--', str(order_date)])
  return(concatenated)

# Let's look at what the function does on a single set of data points
print(concatenate_num_date('foo', 'bar'))
# Now we'll use the function in our lambda function
sales_df['new_order_name_date'] = sales_df.apply(lambda row: concatenate_num_date(row.order_number, row.order_date), axis=1)
sales_df[['order_number', 'order_date', 'new_order_name_date']].head()

In [ ]:
# Bonus: Progress bar
# Use tqdm's progress_apply to render a progress bar during complex / long processing
from tqdm import tqdm # Import the function
tqdm.pandas() # Start tqdm for Pandas operations

# Now, we'll use .progress_apply() instead of the .apply() we have been using
sales_df['new_order_name_date'] = sales_df.progress_apply(lambda row: concatenate_num_date(row.order_number, row.order_date), axis=1)
sales_df[['order_number', 'order_date', 'new_order_name_date']].head()

In [ ]:
# Exercise
'''
Create a new row called "Expensive" with the values True or False. 
1. True if the product cost is >= 2000
2. False if it is less than 2000
'''

##########
## 5.2 CORRELATIONS
##########

In [ ]:
# We can use the .corr() function to compute the pairwise correlation of columns, excluding NA/null values
# By default, the function uses the standard Pearson correlation coefficient
# But we can also choose to use the Kendall Tau coefficient (method=kendall) or the Spearman rank correlation (method=spearman)
# We can also define our own correlation method function, and then use method=function_name
# To learn more: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html

# Here, we'll compute the correlation of all columns in the sales dataframe
corr_matrix = sales_df.corr()
corr_matrix

In [ ]:
# Clearly, it doesn't make much sense to compute correlations between every column
# So let's build a new dataframe with only a few columns, then compute the column correlation again
my_cols = sales_df[['order_quantity', 'product_cost', 'product_price']]
corr_matrix = my_cols.corr()
corr_matrix

##########
## 5.3 MATRIX MULTIPLICATIONS
##########

In [ ]:
# To compute the innner product between two dataframes or series, we can use the .dot() function
# Note: The dimensions of DataFrame and other must be compatible in order to compute the matrix multiplication. 
#       In addition, the column names of DataFrame and the index of other must contain the same values, 
#       as they will be aligned prior to the multiplication.
# Learn more: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dot.html 

# Here, we show a simple example of computing the dot product between two data series
series1 = pd.Series([7, 5, 6, 4, 9]) 
series2 = pd.Series([1, 2, 3, 10, 2]) 
dot_prod = series1.dot(series2) 
dot_prod

########################
# 6. TRANSFORMING DATAFRAMES
########################

In [ ]:
# We can use the .pivot() function to create a new derived dataframe out of a given one 
# The function takes 3 arguements 'index', 'columns', and 'values' - each must be a column name in the original dataframe
# When executed, the function will create a new dataframe, whose row and column indices are the unique values of the 
# respective parameters. The cell values of the new table are taken from column given as the 'values' parameter.

# In this example, we'll be looking at new sales quantity for each brand for each day
# First, we'll isolate the columns we need, then aggregate the new sales quantities

my_cols = sales_df[['order_date', 'product_name', 'order_quantity']]

# new_sales_df = my_cols.groupby(['order_date', 'product_name'], as_index = False).agg('sum')

new_sales_df = my_cols.groupby(['order_date', 'product_name']).agg('sum').reset_index()
new_sales_df.head()

In [ ]:
# Next, we'll create a new table with columns = ProductName, rows = OrderDate, and values = OrderQuantity
result1 = new_sales_df.pivot(index='order_date', columns='product_name', values='order_quantity').fillna(0).astype(int)
result1.head()

In [ ]:
# The .pivot_table() function is a generalization of .pivot() that can handle duplicate values for one pivoted index/column pair 
# Specifically, you can give .pivot_table() a list of aggregation functions using keyword argument aggfunc
# The default aggfunc of .pivot_table() is numpy.mean

# Let's do the same operation we just did, but using .pivot_table() this time
result2 = sales_df.pivot_table(index='order_date', columns='product_name', values='order_quantity', aggfunc=np.sum, fill_value = 0)
result2.head()

In [ ]:
# To confirm that the two results are the same, we can use the .equals() function
equal_bool = result1.equals(result2)
print('The two results are equal:', equal_bool)

In [ ]:
# The .transpose() function is fairly self-explanatory - we can use this function to 
# compute the transpose of a dataframe. Sometimes this comes in handy to get data ready for plotting.
# For example, let's say we need our OrderQuantity pivot table to have columns = dates and rows = products
transposed1 = result2.transpose()
transposed1.head()

In [ ]:
# We can also use df.T to accomplish the same thing (The property T is an accessor to the .transpose() function)
transposed2 = result2.T
print('The two results are equal:', transposed1.equals(transposed2))

In [ ]:
# Exercise
'''
1. Create a new column called 'revenue' which is the product of 'product_price' and 'order_quantity' in a new df called 'new_sales_df'
2. Create a new table with 'order_date' as the index, 'product_name' as column and the value as the aggregate of the revenue values for the product on a date (pivot vs pivot_table)
3. Find the total revenue from AWC Logo Cap in 2016
'''

# new_sales_df = sales_df
# new_sales_df['revenue'] = sales_df.apply(lambda x: x['order_quantity'] * x['product_price'], axis = 1)
# t = new_sales_df.pivot_table(index = 'order_date', columns = 'product_name', values = 'revenue', aggfunc = np.sum, fill_value = 0)
# t
# t.loc['2016-01-01' : '2017-12-31', 'AWC Logo Cap'].sum()

########################
# 7. VISUALIZING DATA
########################

In [ ]:
# Pandas dataframes work very well with another Python library - matplotlib
# For more information about the matplotlib library, see:
# https://matplotlib.org/
# https://www.datacamp.com/community/blog/python-matplotlib-cheat-sheet

# First, we need to import matplotlib
import matplotlib.pyplot as plt

##########
## 7.1 Bar Chart
##########

In [ ]:
# In this example, display the order quantity for each category

# First, we'll prepare the data
data_to_plot = sales_df.groupby('category_name')['order_quantity'].agg('count')
data_to_plot = data_to_plot.reset_index()
data_to_plot

In [ ]:
# Then, we'll make sure our plotting object is cleared
plt.clf()

# Then, we'll create a plot showing the order quantity for each category
ax = data_to_plot.plot.bar(x = 'category_name', y = 'order_quantity')

# Then, we'll set the plot title, x and y axis labels
ax.set_title('Bar chart showing the order quantity of each category')
ax.set_xlabel("Category")
ax.set_ylabel("Order Quantity")

# Then, we'll save the plot into a directory called 'plots/'
plt.savefig('./plots/bar.png')

##########
## 7.2 TIME SERIES
##########

In [ ]:
# In this example, we will plot the daily sales for Road Bikes, Mountain Bikes, Touring Bikes over time

# First, let's look at the data type of our 'order_date' column (this column contains date information)
print(sales_df['order_date'].dtypes) # Result is 'object' meaning the column contains strings

In [ ]:
# In order to plot the data over time, we need to convert the column from a string to a datetime type,
# which can be plotted as a time series. Let's also convert the dates into month bins by setting each
# day to the first of the month, in order to make the visualization easier to interpret.

# We will use a lambda function to do this :)

sales_df['order_date'] = sales_df['order_date'].progress_apply(lambda x: pd.to_datetime(x))

In [ ]:
print(sales_df['order_date'].dtypes)

In [ ]:
bikes_df = sales_df[sales_df['category_name'] == 'Bikes']

In [ ]:
# Next, we'll keep only the columns we need
filtered_df = bikes_df[['order_date', 'subcategory_name', 'order_quantity']]
filtered_df.head()

In [ ]:
filtered_df['subcategory_name'].unique()

In [ ]:
# Next, we'll group first by 'order_date', then by 'subcategory_name', 
# and sum the only remaining column 'order_quantity'
total_bikes_df = filtered_df.groupby(['order_date', 'subcategory_name']).agg('sum').reset_index()
# total_sales_df = total_sales_df.sort_values(by = ['order_date'])
# Let's look at what our data now looks like
total_bikes_df.head()

In [ ]:
total_bikes_df.dtypes

In [ ]:
# Now, we need to pivot the dataframe to get it in the right shape for plotting
pivoted_sales_df = total_bikes_df.pivot(index='order_date', values='order_quantity', columns='subcategory_name')
# Let's look at what our data now looks like
pivoted_sales_df.head()

In [ ]:
pivoted_sales_df = pivoted_sales_df.fillna(0)
pivoted_sales_df.head()

In [ ]:
# Finally, we're ready to plot
# We clear the plot area, plot our data on the axis, add labels, and save the figure
plt.clf()
ax = pivoted_sales_df.plot()
ax.set_title('Total weekly sales for Road Bikes, Mountain Bikes, Touring Bikes')
ax.set_xlabel("Week beginning")
ax.set_ylabel("Total sales")
plt.savefig('plots/time_series.png')

In [ ]:
# An easier to visualize way to plot this data

pivoted_sales_df = pivoted_sales_df.cumsum(axis = 0)
pivoted_sales_df.head()

In [ ]:
# Finally, we're ready to plot
# We clear the plot area, plot our data on the axis, add labels, and save the figure
plt.clf()
ax = pivoted_sales_df.plot()
ax.set_title('Total weekly sales for Road Bikes, Mountain Bikes, Touring Bikes')
ax.set_xlabel("Week beginning")
ax.set_ylabel("Total sales")
plt.savefig('plots/time_series.png')

##########
## 7.3 HEAT MAP
##########

In [ ]:
# Another plotting option is a heat map 

# We import a supporting library and prepare the plot area
import seaborn as sns
plt.clf()
plt.figure(figsize=(20,10)) # This explicity sets the size of the plot area (2x as wide as tall)
ax = plt.axes()

# Then we plot the data, add labels, and save the figure
sns.heatmap(pivoted_sales_df, ax = ax)
ax.set_title('Heatmap of weekly sales for Road Bikes, Mountain Bikes, Touring Bikes')
ax.set_xlabel("Bike Type")
ax.set_ylabel("Week beginning")
plt.savefig('plots/heatmap.png')

##########
## 7.4 SCATTER PLOT
##########

In [ ]:
# Let's plot month vs total sales amount 
new_sales_df = sales_df
new_sales_df['revenue'] = sales_df.apply(lambda x: x['order_quantity'] * x['product_price'], axis = 1)
# new_sales_df['revenue'] = sales_df.apply(lambda x: , axis = 1)

# Let's see if the month has an effect on the amount of revenue
rev = new_sales_df.groupby(new_sales_df.order_date.dt.month).agg('revenue').sum().reset_index()
rev

In [ ]:
fig, ax = plt.subplots()
ax.plot(rev.order_date, rev.revenue, marker='o', linestyle='', ms=12, alpha = 0.5)
plt.savefig('plots/scatter_plot.png')

##########
## 7.5 SCATTER PLOT WITH NICER FORMATTING
##########

In [ ]:
# Matplotlib allows for an extensive amount of formatting customization
# Options include, but are certainly not limited to: graph title, axis titles, grid, axis range, legend position
fig, ax = plt.subplots()

# And axis (data series) title
ax.set_title('Monthly sales overview')
# Make the grid visible
ax.grid(True)
# Set the x axis label and range
ax.set_xlabel('Month')
ax.set_xlim((0, 15))
# Set the y axis label and range
ax.set_ylabel('Total sales value')
ax.set_ylim((0, 5000000))

ax.plot(rev.order_date, rev.revenue, marker='o', linestyle='', ms=12, alpha = 0.5)

# Now we'll save the new version and compare to the previous one
plt.savefig('plots/scatter_plot_new.png')

########################
# 8. ACTIVITIES
########################

In [ ]:
customer_df = pd.read_csv('./data/customers_data.csv')

# Let's switch to the customers dataset for the activities. First we have to turn the annual_income
# column from a type string into a type integer
def salary_transform(string):
    s = string.replace('$', '')
    s = s.replace(',', '')
    return int(s)

customer_df['annual_income'] = customer_df['annual_income'].apply(lambda x: salary_transform(x))
customer_df.head()

##########
## 8.1 ACTIVITY 1
##########

In [ ]:
# Using the customer_df:
# 1. Which 'occupation' has the HIGHEST MEAN and MEDIAN 'annual_income' value? And what are the values? 
# Hint: it's the same occupation.

# 2. For the occupation you found in step 1, how many rows in customer_df does it appear in?

###
# YOUR CODE HERE


###

# Correct results for step 1: occupation = Management, mean = 92118.53, median = 90000
# Correct results for step 2: 3011 rows

##########
## 8.2 ACTIVITY 2
##########

In [ ]:
sales_df = pd.read_csv('./data/sales_data_all.csv')

In [ ]:
# Using the sales_df:
# 1. Use a lambda function to compute ('product_price' - 'product_cost') for each row and store the value in 
#    a new column called 'delta'
#    Then, sum all of the resulting values in 'delta' to compare with the correct answer below
# 
# 2. Use a lambda function and subsequent aggregation to compute how many more characters, 
#    on average (mean), the 'product_description' column has compared to the 'product_name' 

###
# YOUR CODE HERE


###

# Correct answer for step 1: 10268688.964499999
# Correct answer for step 2: 55.49814438140099